## Write predictions to the database

In [11]:
import psycopg2
import pandas as pd
import numpy as np

In [ ]:
# Conect to the database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="")
cur = conn.cursor()

In [ ]:
# Set the table names
parcels_table = "aoi2020"
bs_tensorflow = "bs_tensorflow"

# Set the folder to store the data
data_folder = ''

In [ ]:

insertSql = "INSERT into {} (pid, class, {}) values ({}, {}, {})"

updateSql = "UPDATE {} set {} = {} WHERE pid = {}"

nchunks = 4

index = pd.DataFrame().index

for n in range(nchunks):
    df = pd.read_csv(f'{data_}{parcels_table}_predictions_{n}.csv', index_col = 0, low_memory=False)
    r_index = df.columns[1:]

    df[f'pred{n}'] = df.apply(lambda x: np.array(x[r_index]).argmax(), axis=1)

    df.drop(r_index, axis=1, inplace=True)
    
    df = df[~df.index.duplicated(keep='last')]
    print(n, len(df))
    
    df_new = df.loc[set(df.index) - set(index)].copy()
    df_old = df.loc[index].copy()
    df_old.dropna(inplace=True)

    if len(df_new):
        for i in df_new.index:
            r = df_new.loc[i]
            cur.execute(insertSql.format(bs_tensorflow, f'p{n}', i, r[0], r[1]))
    
    conn.commit()
    
    if len(df_old):
        for i in df_old.index:
            r = df_old.loc[i]
            #print(r)
            cur.execute(updateSql.format(bs_tensorflow, f'p{n}', r[1], i))
    
    conn.commit()
    
    index = index.append(df_new.index)
    
    print(n, len(index))



In [ ]:
# Close database connection
database.close_connection()